<a href="https://colab.research.google.com/github/Alenushka2013/Coursera/blob/master/214747_4_7_Data_leak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import pandas as pd

# Чтение данных
data = pd.read_csv('AER_credit_card_data.csv',
                   true_values = ['yes'], false_values = ['no'])

# Выбор целевой переменной
y = data.card

# Выбор предикторов
X = data.drop(['card'], axis=1)

print("Number of rows in the dataset:", X.shape[0])
X.head()

Number of rows in the dataset: 1319


,reports,age,income,share,expenditure,owner,selfemp,dependents,months,majorcards,active
0,0,37.66667,4.5200,0.033270,124.983300,True,False,3,54,1,12
1,0,33.25000,2.4200,0.005217,9.854167,False,False,3,34,1,13
2,0,33.66667,4.5000,0.004156,15.000000,True,False,4,58,1,5
3,0,30.50000,2.5400,0.065214,137.869200,False,False,0,25,1,7
4,0,32.16667,9.7867,0.067051,546.503300,True,False,2,64,1,5


Используем перекрестную проверку, чтобы обеспечить точные показатели качества модели.

In [29]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Поскольку предварительная обработка не требуется, нам не нужен pipeline
my_pipeline = make_pipeline(RandomForestClassifier(n_estimators=100))
cv_scores = cross_val_score(my_pipeline, X, y,
                            cv=5,
                            scoring='accuracy')

print("Cross-validation accuracy: %f" % cv_scores.mean())

Cross-validation accuracy: 0.979537


**Несколько подозрительно!**

Ниже приведено краткое изложение данных, с которыми мы работаем:

    card: 1, если заявка на кредитную карту принята, 0, если нет
    reports: Количество крупных уничижительных отчетов
    age: Возраст n лет плюс двенадцатые части года
    income: Годовой доход (деленный на 10 000)
    share: Соотношение ежемесячных расходов по кредитной карте к годовому доходу
    expenditure: Средние ежемесячные расходы по кредитной карте/картам
    owner: 1, если владеет домом, 0 - если арендует
    selfempl: 1, если самозанятый, 0 - если нет
    dependents: 1 + количество иждивенцев
    months: Месяцы проживания по текущему адресу
    majorcards: Количество основных кредитных карт, имеющихся в наличии
    active: Количество активных кредитных счетов

Несколько переменных выглядят подозрительно! Например, expenditure означает ли это расходы по этой карте или по картам, которые использовались до подачи заявления?

Аналогично, переменная share(доля расходов по кредитной карте от годового дохода) может быть индикатором будущих расходов и, следовательно, утечкой цели.

На этом этапе могут оказаться весьма полезными базовые сравнения данных:

In [30]:
expenditures_cardholders = X.expenditure[y]
expenditures_noncardholders = X.expenditure[~y]

print('Fraction of those who did not receive a card and had no expenditures: %.2f' \
      %((expenditures_noncardholders == 0).mean()))
print('Fraction of those who received a card and had no expenditures: %.2f' \
      %(( expenditures_cardholders == 0).mean()))

Fraction of those who did not receive a card and had no expenditures: 1.00
Fraction of those who received a card and had no expenditures: 0.02


Как показано выше, доля людей, которые не получили карту и не имели расходов, составляет 100%. В то же время среди тех, кто получил карту, только 2% не сделали ни одной покупки. Неудивительно, что наша модель показала высокую точность, которую можно объяснить именно этой тенденцией. Однако данный факт также представляет собой пример утечки цели, так как расходы, скорее всего, относятся к расходам по карте, на которую они подавали заявку.

    Модель показала высокую точность, потому что среди тех, кто не получил карту, 100% не имели расходов, а среди получивших карту только 2% не потратили ни копейки. Это создало ложное впечатление о высокой эффективности модели, поскольку она могла легко предсказать отсутствие расходов у тех, кто не получил карту.

Поскольку уровень расходов (expenditure) влияет на результаты модели, эту переменную следует исключить из анализа. Переменная share, также, потенциально содержит информацию о целевой переменной и должна быть удалена. Переменные "active" и "majorcards" менее очевидны, но их описание вызывает определенные опасения. В большинстве случаев лучший подход — это перестраховка. Если у Вас нет возможности выяснить, кто создавал данные, лучше проявить осторожность, чем потом сожалеть о возможных последствиях.

**Модель без целевой утечки**

Запустим модель без целевой утечки следующим образом:

In [31]:
# Удалить потенциально целевую утечку из набора данных
potential_leaks = ['expenditure', 'share', 'active', 'majorcards']
X2 = X.drop(potential_leaks, axis=1)

# Оценить модель после удаления потенциально целевой утечки
cv_scores = cross_val_score(my_pipeline, X2, y,
                            cv=5,
                            scoring='accuracy')

print("Cross-val accuracy: %f" % cv_scores.mean())

Cross-val accuracy: 0.830165



**Заключение**

Утечка данных может быть ошибкой стоимостью в несколько миллионов долларов во многих проектах/решениях науки о данных. Тщательное разделение данных обучения и проверки может предотвратить загрязнение данных обучения и тестирования, а конвейеры могут помочь реализовать это разделение. Аналогично, сочетание осторожности, здравого смысла и исследования данных может помочь выявить целевую утечку!
